# Poisson vs Linear Regression: Toronto Crime Data 2021
---
This notebook compares **Poisson regression** and **Linear regression** models using crime and income data from Toronto neighborhoods.

We:
- Merge crime and income datasets
- Create total crime rate per 100,000 population
- Fit Poisson regression and Linear regression models
- Visualize model coefficients and predictions
- Compare model fits visually and with interpretation


In [ ]:
import pandas as pd

# Load datasets
crime_df = pd.read_csv("cleaned_crime_data_2021.csv")
income_df = pd.read_csv("cleaned_neighbourhood_income_data_2021.csv")

# Merge on neighborhood ID
merged_df = pd.merge(crime_df, income_df, left_on="HOOD_ID", right_on="neighbourhood_id", how="inner")
merged_df.dropna(inplace=True)

# Create total crime rate per 100,000 population
merged_df["TOTAL_CRIME_RATE"] = (merged_df["TOTAL_CRIME_COUNT"] / merged_df["total_population"]) * 100000


In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Formulas
formula_count = """
TOTAL_CRIME_COUNT ~ median_income + average_income + median_after_tax_income +
average_after_tax_income + market_income_percent + employment_income_percent +
government_support_percent + low_income_percent + percent_without_income +
median_to_average_ratio + is_improvement_area
"""

formula_rate = """
TOTAL_CRIME_RATE ~ median_income + average_income + median_after_tax_income +
average_after_tax_income + market_income_percent + employment_income_percent +
government_support_percent + low_income_percent + percent_without_income +
median_to_average_ratio + is_improvement_area
"""

# Fit Poisson models
model_count = smf.glm(formula=formula_count, data=merged_df, family=sm.families.Poisson()).fit()
model_rate = smf.glm(formula=formula_rate, data=merged_df, family=sm.families.Poisson()).fit()

model_count.summary(), model_rate.summary()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def extract_coefficients(model, label):
    summary = model.summary2().tables[1]
    summary = summary.reset_index().rename(columns={"index": "Variable"})
    summary["Model"] = label
    return summary[["Variable", "Coef.", "[0.025", "0.975]", "Model"]]

coef_count = extract_coefficients(model_count, "Crime Count")
coef_rate = extract_coefficients(model_rate, "Crime Rate")
combined = pd.concat([coef_count, coef_rate])

# Plot
plt.figure(figsize=(12, 8))
sns.set(style="whitegrid")
ax = sns.pointplot(data=combined, y="Variable", x="Coef.", hue="Model", dodge=0.5, join=False, ci=None)
for i, row in combined.iterrows():
    plt.plot([row["[0.025"], row["0.975]"]], [i, i], color="black", linewidth=1)
plt.axvline(0, color='gray', linestyle='--')
plt.title("Poisson Regression Coefficients with 95% CI")
plt.xlabel("Coefficient Estimate")
plt.ylabel("Predictor Variables")
plt.legend()
plt.tight_layout()
plt.gca().invert_yaxis()
plt.show()


In [ ]:
# Linear regression
predictors = [
    'median_income', 'average_income', 'median_after_tax_income',
    'average_after_tax_income', 'market_income_percent',
    'employment_income_percent', 'government_support_percent',
    'low_income_percent', 'percent_without_income',
    'median_to_average_ratio', 'is_improvement_area'
]
X_lin = sm.add_constant(merged_df[predictors])
y_count = merged_df['TOTAL_CRIME_COUNT']
linear_model = sm.OLS(y_count, X_lin).fit()

# Poisson predictions
merged_df["predicted_count"] = model_count.predict(merged_df)
merged_df["predicted_rate"] = model_rate.predict(merged_df)

# Comparison plot (fit vs fit)
fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(linear_model.fittedvalues, merged_df['predicted_count'], s=20)
ax.set_xlabel('Linear Regression Fit', fontsize=20)
ax.set_ylabel('Poisson Regression Fit', fontsize=20)
ax.axline([0, 0], slope=1, c='black', linewidth=3, linestyle='--')
plt.tight_layout()
plt.show()


## 🧠 Model Fit Interpretation

- **Points on the diagonal line**: Agreement between Poisson and Linear models.
- **Points below the line**: Poisson predicts lower than Linear — especially for high crime neighborhoods.
- **Poisson Regression** is preferred for count data:
  - Handles skew and non-constant variance
  - Uses a log link, preventing unrealistic predictions
  - Deviance and Log-Likelihood indicate strong fit
- In this case, **Poisson regression is the better model**.

Further improvements: test for overdispersion or try a Negative Binomial model.
